<a href="https://colab.research.google.com/github/mahii6991/transfer_learning_bert/blob/main/fine_tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
 !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-05-29 22:52:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-05-29 22:52:28 (75.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [2]:
#here we will be coding a feature based , or working on the feature based model and understanding its accuracy with respect to our dataset
from helper_functions import unzip_data, calculate_results
unzip_data('/content/train_test.zip')

# Loading Dataset

In [4]:
import pandas as pd
df_train = pd.read_csv('/content/train.csv')
df_train.head()

,Unnamed: 0,sentences,Subject,Object,relations,relations_id
0,0,“Brands that optimized their eGift and gift ca...,Theresa McEndree,Blackhawk Network,managerOf,0
1,1,"This is a huge win for retailers, as gift car...","Theresa McEndree,",Blackhawk Network,managerOf,0
2,4,"This is a huge win for retailers, as gift car...",Theresa McEndree,Blackhawk Network,employedBy,2
3,5,by[E1] Esther Shein[/E1] in [E2]CXO[/E2],Esther Shein,CXO,employedBy,2
4,6,COVID-19 has forced many organizations to rein...,Claudia Crummenerl,Capgemini Invent,managerOf,0


In [6]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00


In [11]:
#improting the training file into the system
training = pd.read_csv("/content/train.csv")
#improting the testing file after getting the zip file
testing = pd.read_csv("/content/test.csv")

In [12]:
#need to create the training file and the testing file 
X_train = training['sentences']
y_train = training['relations_id']

#creating the testing file of the model
X_test = testing['sentences']
y_test = testing['relations_id']

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the training file
df_train = pd.read_csv("train.csv")

# Split the training data into train and validation sets
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

# Save the validation data to a CSV file
val_df.to_csv("val.csv", index=False)


In this code the first 11 layers of the bert model is freezed and only the last layer is used , this is a very basic code to fine tune the bert model and will see how more changes can be done in the future.

In [19]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


In [20]:
from datasets import load_dataset

# Load the dataset
annotated_dataset = load_dataset("csv", data_files={
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv",
})

# Access the train, validation, and test splits
train_dataset = annotated_dataset["train"]
val_dataset = annotated_dataset["validation"]
test_dataset = annotated_dataset["test"]


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0239c8e2fcec4a46/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 814
})

In [22]:
val_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 163
})

In [23]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'sentences', 'Subject', 'Object', 'relations', 'relations_id'],
    num_rows: 204
})

In [24]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

# # Load the dataset
# annotated_dataset = load_dataset("csv", data_files={
#     "train": "train.csv",
#     "validation": "val.csv",
#     "test": "test.csv",
# })

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
annotated_dataset_tokenized = annotated_dataset.map(tokenizer, batched=True, batch_size=None)
#del imdb_dataset
annotated_dataset_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Create a DataLoader for the training set
train_loader = DataLoader(
    dataset=imdb_tokenized["train"],
    batch_size=12,
    shuffle=True, 
    num_workers=4
)

# Create a DataLoader for the validation set
val_loader = DataLoader(
    dataset=imdb_tokenized["validation"],
    batch_size=12,
    num_workers=4
)

# Create a DataLoader for the test set
test_loader = DataLoader(
    dataset=imdb_tokenized["test"],
    batch_size=12,
    num_workers=4
)

# Create a model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Create a training loop
trainer = Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=100,
)

# Train the model
trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

# Evaluate the model
trainer.test(model=lightning_model, dataloaders=test_loader, ckpt_path="best")


Map:   0%|          | 0/814 [00:00<?, ? examples/s]

ValueError: ignored